In [12]:
import pandas as pd
import numpy as np

chronological_age = 9.8
age_ranges = np.arange(0, 100, 5)

index_in_age_ranges = np.searchsorted(age_ranges, chronological_age)

print(index_in_age_ranges)
age_ranges[index_in_age_ranges]

2


10

In [1]:
a = ["a", "x", "d"]

In [2]:
a.sort()

In [3]:
a

['a', 'd', 'x']